In [28]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from tqdm import tqdm

In [29]:
# get list of urls for players stats
players_url = 'https://www.basketball-reference.com/players/'
r = requests.get(players_url)
bs = BeautifulSoup(r.text)

In [31]:
# get all last name directory links
    # links found in 'ul' tag with 'page_index' class
    # directory links will have a text with length 1 (e.g. "A")
    # base url must be added to directory path
letter_links = bs.find('ul', {'class': 'page_index'}).find_all('a')
letter_links = ['https://www.basketball-reference.com' + link.get('href') for link in letter_links if len(link.text) == 1]
print(len(letter_links))

25


In [32]:
# get all player links from last name directory links

# initialize empty list of player links
player_links = []

# player links found in 'th' tag w/specific scope and class
# active players with 'strong' tag
# add base url
for letter_link in letter_links:
    r = requests.get(letter_link)
    bs = BeautifulSoup(r.text)
    player_tags = bs.find('tbody').find_all('th', {'scope': 'row', 'class': 'left'})
    player_links += ['https://www.basketball-reference.com' + player_tag.find('a').get('href') for player_tag in player_tags]

print(len(player_links))

5087


In [33]:
# to break up requests
import time

In [34]:
# build dataframe of all players' stats by season

# initialize empty list to which dfs will be appended
appended_data = []


for player_link in tqdm(player_links):
    time.sleep(5)
    # get tables from player page
    dfs = pd.read_html(player_link)
    # save df as second table in html table code
    df = dfs[1]

    # add player name as col
    r = requests.get(player_link)
    bs = BeautifulSoup(r.text)
    name = bs.find("div", {"id": "info", "class": "players"}).find("h1").text
    name = name.split('\n')[1]

    df['Name'] = name
    # add years of experience (use index + 1)
    df['Exp'] = df.index + 1

    # append individual df to list
    appended_data.append(df)

# combine all dfs together
nba = pd.concat(appended_data)

  0%|          | 2/5087 [00:04<2:59:20,  2.12s/it]


HTTPError: HTTP Error 429: Too Many Requests

In [145]:
# move Exp and Name to be beginning cols
first = nba.pop('Exp')
nba.insert(0, 'Exp', first)

first = nba.pop('Name')
nba.insert(0, 'Name', first)

# drop all rows where Age is NaN (these are not specific years, could be career or specific team average)
nba = nba.dropna(subset=['Age'])

# reset index
nba = nba.reset_index(drop=True)

In [146]:
nba.head()

,Name,Exp,Season,Age,Tm,Lg,Pos,G,GS,MP,...,FT%,ORB,DRB,TRB,AST,STL,BLK,TOV,PF,PTS
0,Álex Abrines,1,2016-17,23.0,OKC,NBA,SG,5,0,16.0,...,0.75,0.4,1.4,1.8,0.8,0.0,0.0,0.0,1.6,4.8
1,Álex Abrines,2,2017-18,24.0,OKC,NBA,SG,6,0,18.3,...,1.0,0.5,2.2,2.7,0.3,0.8,0.3,0.2,1.8,4.0
2,Precious Achiuwa,1,2020-21,21.0,MIA,NBA,PF,61.0,4.0,12.1,...,0.509,1.2,2.2,3.4,0.5,0.3,0.5,0.7,1.5,5.0
3,Precious Achiuwa,2,2021-22,22.0,TOR,NBA,C,73.0,28.0,23.6,...,0.595,2.0,4.5,6.5,1.1,0.5,0.6,1.2,2.1,9.1
4,Precious Achiuwa,3,2022-23,23.0,TOR,NBA,PF,1.0,0.0,18.0,...,0.5,1.0,4.0,5.0,0.0,0.0,0.0,0.0,1.0,10.0


In [148]:
nba.shape

(4637, 32)

In [150]:
# check to see # unique players, should be 859
nba.Name.nunique()

859

In [151]:
# write nba df to csv
nba.to_csv('nba_all.csv')